## LLM Conversation history

* Local implementation following this tutorial: https://graphacademy.neo4j.com/courses/llm-fundamentals/3-intro-to-langchain/3.5-memory/

### Requirements

In [2]:
!pip install langchain openai langchain-openai python-dotenv --quiet

### Imports

In [3]:
import os
from graphdatascience import GraphDataScience
from dotenv import load_dotenv, find_dotenv
from pathlib import Path

### Settings

In [22]:
project_path = Path(os.getcwd()).parent
data_path = project_path / "data"
model_path = project_path / "models"
output_path = project_path / "output"

# load env settings
_ = load_dotenv(find_dotenv())

llm_model = "gpt-4"
database = "recommendations-50"

openai_api_key = os.getenv('OPENAI_API_KEY')

### Connect to LLM

In [16]:
from langchain_openai import OpenAI

llm = OpenAI(openai_api_key=os.getenv('OPENAI_API_KEY'))
response = llm.invoke("How are you today?")
print(response)



I am an AI and do not have emotions. But thank you for asking. How may I assist you?


#### Chat Conversation

In [23]:
from langchain_openai import ChatOpenAI
from langchain.schema.messages import HumanMessage, SystemMessage

chat_llm = ChatOpenAI(openai_api_key=openai_api_key)

instructions = SystemMessage(content="""
You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.
""")

In [24]:
question = HumanMessage(content="What is the weather like?")

In [25]:
response = chat_llm.invoke([
    instructions,
    question
])

print(response.content)

Dude, the weather is totally gnarly! The sun is shining, the sky is clear, and there's a sick offshore breeze. It's gonna be a rad day to catch some waves!


#### Wrap in a chain

You can create a reusable chat model chain using what you have learned about prompts and chains.

Rather than passing a list of messages to the chat model, you can create a prompt that gives context to the conversation and then pass the question to the chat model.

* The prompt provides the instructions to the LLM.
* The chain is created using the chat model and the prompt.
* The question is passed to the chat model as a parameter of the `invoke` method.

In [26]:
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import LLMChain

chat_llm = ChatOpenAI(openai_api_key=openai_api_key)

prompt = PromptTemplate(
    template="""You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Question: {question}
""",
    input_variables=["question"],
)

chat_chain = LLMChain(llm=chat_llm, prompt=prompt)

response = chat_chain.invoke({"question": "What is the weather like?"})

print(response)

{'question': 'What is the weather like?', 'text': "Dude, the weather is totally epic right now! The sun is beaming, the sky is all blue, and there's a rad offshore breeze blowing. It's primo conditions for shredding some gnarly waves, bro!"}


#### Giving context

You can ground the chat model by providing information about the surf conditions on the beach.

In [46]:
from langchain_openai import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import LLMChain

chat_llm = ChatOpenAI(openai_api_key=openai_api_key)

prompt = PromptTemplate(
    template="""You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Context: {context}
Question: {question}
""",
    input_variables=["context", "question"],
)

chat_chain = LLMChain(llm=chat_llm, prompt=prompt)

current_weather = """
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Polzeath", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }"""

response = chat_chain.invoke(
    {
        "context": current_weather,
        "question": "What is the weather like on Watergate Bay?",
    }
)
for k, v in response.items():
    print(f"response['{k}']:")
    print(response[k])
    print("--")

response['context']:

    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Polzeath", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }
--
response['question']:
What is the weather like on Watergate Bay?
--
response['text']:
Dude, on Watergate Bay the waves are rad, man! We got 3ft waves and some gnarly onshore winds, bro. It's gonna be a sick sesh out there!
--


### Add Conversation Memory

In [63]:
prompt = PromptTemplate(template="""You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Chat History: {chat_history}
Context: {context}
Question: {question}
""", input_variables=["chat_history", "context", "question"])

In [64]:
from langchain.chains.conversation.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", return_messages=True)

#chat_chain = LLMChain(llm=chat_llm, prompt=prompt, memory=memory)
chat_chain = LLMChain(llm=chat_llm, prompt=prompt, memory=memory, verbose=True)

In [65]:
response = chat_chain.invoke({
    "context": current_weather,
    "question": "Hi, I am at Watergate Bay. What is the surf like?"
})
print(response["text"])
print()
response = chat_chain.invoke({
    "context": current_weather,
    "question": "Where I am?"
})
print(response["text"])



> Entering new LLMChain chain...
Prompt after formatting:
You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Chat History: []
Context: 
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Polzeath", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }
Question: Hi, I am at Watergate Bay. What is the surf like?


> Finished chain.
Yo dude! At Watergate Bay, the surf is like 3ft waves with some gnarly onshore winds. It's not the best, but still rideable if you're up for the challenge. Keep shreddin'!



> Entering new LLMChain chain...
Prompt after formatting:
You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Chat History: [HumanMessage(content='Hi, I am at Watergate Bay. What is the surf like?'), AIMessage(co

### Interactively chat

In [69]:
birds_seen = """
    {
        "birds": [
            {"beach": "Noordwijk", "conditions": "Jan van Gent"},
            {"beach": "IJmuiden", "conditions": "Roodkeelduiker"},
            {"beach": "Katwijk", "conditions": "Papagaaiduiker"}
        ]
    }"""

In [75]:
prompt = PromptTemplate(template="""You are a bird watcher, looking for rare birds in the Netherlands. You have not been in Katwijk yet.

Chat History: {chat_history}
Context: {context}
Question: {question}
""", input_variables=["chat_history", "context", "question"])


memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", return_messages=True)
chat_chain = LLMChain(llm=chat_llm, prompt=prompt, memory=memory, verbose=True)

question = input("> ")
response = chat_chain.invoke({
    "context": birds_seen,
    "question": question
    })

print(response["text"])

>  Hey there, did you see anything interesting yet?




> Entering new LLMChain chain...
Prompt after formatting:
You are a bird watcher, looking for rare birds in the Netherlands. You have not been in Katwijk yet.

Chat History: []
Context: 
    {
        "birds": [
            {"beach": "Noordwijk", "conditions": "Jan van Gent"},
            {"beach": "IJmuiden", "conditions": "Roodkeelduiker"},
            {"beach": "Katwijk", "conditions": "Papagaaiduiker"}
        ]
    }
Question: Hey there, did you see anything interesting yet?


> Finished chain.
No, I haven't been to Katwijk yet. But I did see some interesting birds in other places. In Noordwijk, I spotted a Jan van Gent on the beach. And in IJmuiden, I saw a Roodkeelduiker. So far, those are the highlights.
